In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
#from keras.datasets import mnist
from keras import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam,SGD
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
import glob
from skimage import io
import os
import scipy.misc
from scipy.misc import imread, imresize
from keras import regularizers
import csv
from keras.applications.inception_v3 import InceptionV3, preprocess_input

Using TensorFlow backend.


In [2]:

datasets_path = 'C:/Users/Pushkar/Downloads/Datasets/101_ObjectCategories' #Add the path to the unzipped folder

def load_images(path,n=0):
    X = []
    Y=[]
    i=-1
    labels = []
    for label in os.listdir(path):
        back_path = os.path.join(path,label)
        labels.append(label)
        i = i+1
        for filename in os.listdir(back_path):
            image_path = os.path.join(back_path,filename)
            img = image.load_img(image_path,target_size=(224,224))
            img = image.img_to_array(img)
            #Y.append(image)
            #image = imresize(image,[128,128,3])
            #image = imresize(imread(image_path), [128,128, 3])
            #image = image.astype('float32')
            img[:,:,0] -= 123.68
            img[:,:,1] -= 116.78
            img[:,:,2] -= 103.94
            #image = image/255
            #image = 1-image
            Y.append(i)
            X.append(img)
            #X.append(image.img_to_array(img))
            #X.append(image)
    return X,Y,labels

x_train,y,label_data = load_images(datasets_path)
X = np.array(x_train)
#print(len(x_train))
#print(x_train.shape)
Y = np.array(y)
print(X.shape)

(8677, 224, 224, 3)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [4]:
print(y_train.shape)
print(X_train.shape)

(6941,)
(6941, 224, 224, 3)


In [5]:
number_of_classes = 101
Y_train = np_utils.to_categorical(y_train, number_of_classes)
Y_test = np_utils.to_categorical(y_test, number_of_classes)


In [6]:
gen = ImageDataGenerator(width_shift_range=.2, 
                             height_shift_range=.2,
                          zoom_range=0.2)

test_gen = ImageDataGenerator()
train_generator = gen.flow(X_train, Y_train, batch_size=16)
test_generator = test_gen.flow(X_test, Y_test, batch_size=16)

In [7]:
from keras.applications. vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

In [8]:
def add_new_last_layer(base_model, nb_classes):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(nb_classes, activation='softmax')(x)
    model = Model(input=base_model.input, output=predictions)
    return model

In [9]:
base_model = ResNet50(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
model = add_new_last_layer(base_model, 101)

C:\Users\Pushkar\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [11]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(lr = 1e-3,momentum = 0.9),metrics=['accuracy'])

In [12]:
x = model.fit_generator(train_generator, epochs=1, 
                    validation_data=test_generator)

Epoch 1/1
434/434 [==============================] - 261s 600ms/step - loss: 1.6768 - acc: 0.6517 - val_loss: 0.3126 - val_acc: 0.9124


In [13]:
x = model.fit_generator(train_generator, epochs=1, shuffle = True,
                    validation_data=test_generator)

Epoch 1/1
434/434 [==============================] - 257s 591ms/step - loss: 0.3047 - acc: 0.9239 - val_loss: 0.2299 - val_acc: 0.9384


In [14]:
x = model.fit_generator(train_generator, epochs=1, shuffle = True,
                    validation_data=test_generator)

Epoch 1/1
434/434 [==============================] - 259s 597ms/step - loss: 0.1303 - acc: 0.9675 - val_loss: 0.1841 - val_acc: 0.9522


In [15]:
x = model.fit_generator(train_generator, epochs=1, 
                    validation_data=test_generator)

Epoch 1/1
434/434 [==============================] - 260s 600ms/step - loss: 0.0854 - acc: 0.9777 - val_loss: 0.1572 - val_acc: 0.9510


In [16]:
score = model.evaluate(X_test, Y_test)
print()
print('Test loss: ', score[0])
print('Test Accuracy', score[1])


1736/1736 [==============================] - 24s 14ms/step

Test loss:  0.157152753077
Test Accuracy 0.951036866359


In [17]:
print(label_data)

['accordion', 'airplanes', 'anchor', 'ant', 'barrel', 'bass', 'beaver', 'binocular', 'bonsai', 'brain', 'brontosaurus', 'buddha', 'butterfly', 'camera', 'cannon', 'car_side', 'ceiling_fan', 'cellphone', 'chair', 'chandelier', 'cougar_body', 'cougar_face', 'crab', 'crayfish', 'crocodile', 'crocodile_head', 'cup', 'dalmatian', 'dollar_bill', 'dolphin', 'dragonfly', 'electric_guitar', 'elephant', 'emu', 'euphonium', 'ewer', 'Faces', 'Faces_easy', 'ferry', 'flamingo', 'flamingo_head', 'garfield', 'gerenuk', 'gramophone', 'grand_piano', 'hawksbill', 'headphone', 'hedgehog', 'helicopter', 'ibis', 'inline_skate', 'joshua_tree', 'kangaroo', 'ketch', 'lamp', 'laptop', 'Leopards', 'llama', 'lobster', 'lotus', 'mandolin', 'mayfly', 'menorah', 'metronome', 'minaret', 'Motorbikes', 'nautilus', 'octopus', 'okapi', 'pagoda', 'panda', 'pigeon', 'pizza', 'platypus', 'pyramid', 'revolver', 'rhino', 'rooster', 'saxophone', 'schooner', 'scissors', 'scorpion', 'sea_horse', 'snoopy', 'soccer_ball', 'stapler